In [14]:
sequence_key = 'p7s1'
camera_number = 1
calibraton_file_path_base = "gait3d\\Sequences\\{sequence_key}\\Calibration\\c{camera_number}.xml"
calibraton_file_path_1 = calibraton_file_path_base.format(sequence_key=sequence_key, camera_number=1)
print(calibraton_file_path_1)

calibraton_file_path_2 = calibraton_file_path_base.format(sequence_key=sequence_key, camera_number=2)
print(calibraton_file_path_2)

gait3d\Sequences\p7s1\Calibration\c1.xml
gait3d\Sequences\p7s1\Calibration\c2.xml


In [2]:
with open(calibraton_file_path) as file:
    xml_calibration_config = file.read()
    print(xml_calibration_config)

<?xml version="1.0" encoding="UTF-8"?>
<Camera name="c1">
	<Geometry width="960" height="540" ncx="9.600000e+002" nfx="9.600000e+002" dx="1.000000e+000" dy="1.000000e+000" dpx="1.000000e+000" dpy="1.000000e+000"/>
	<Intrinsic focal="5.561470e+002" kappa1="2.720130e-007" cx="4.800000e+002" cy="2.700000e+002" sx="1.001630e+000"/>
	<Extrinsic tx="2.564550e+001" ty="6.130300e+002" tz="3.444080e+003" rx="-3.121731e+000" ry="1.353192e+000" rz="1.582054e+000"/>
</Camera>



In [52]:
import numpy as np
import cv2
from xml.etree import ElementTree as ET

def parse_camera_params(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    intrinsic = root.find("Intrinsic")
    extrinsic = root.find("Extrinsic")
    
    focal = float(intrinsic.get("focal"))
    cx = float(intrinsic.get("cx"))
    cy = float(intrinsic.get("cy"))
    sx = float(intrinsic.get("sx"))
    
    K = np.array([[focal * sx, 0, cx],
                  [0, focal, cy],
                  [0, 0, 1]])
    
    tx = float(extrinsic.get("tx"))
    ty = float(extrinsic.get("ty"))
    tz = float(extrinsic.get("tz"))
    rx = float(extrinsic.get("rx"))
    ry = float(extrinsic.get("ry"))
    rz = float(extrinsic.get("rz"))
    
    R, _ = cv2.Rodrigues(np.array([rx, ry, rz]))
    t = np.array([[tx], [ty], [tz]])

    return K, R, t

def triangulate_points(K1, R1, t1, K2, R2, t2, points1, points2):
    P1 = K1 @ np.hstack((R1, t1))
    P2 = K2 @ np.hstack((R2, t2))
    
    points1_h = cv2.convertPointsToHomogeneous(points1).reshape(-1, 3).T
    points2_h = cv2.convertPointsToHomogeneous(points2).reshape(-1, 3).T
    print(f"{P1 = } \n {points1_h = }")
    print(f"{P2 = } \n {points2_h = }")
    
    points_4d = cv2.triangulatePoints(P1, P2, points1_h[:2], points2_h[:2])
    print(f"{points_4d= }")
    points_3d = points_4d[:3] / points_4d[3]
    return points_3d.T

K1, R1, t1 = parse_camera_params(calibraton_file_path_1)
K2, R2, t2 = parse_camera_params(calibraton_file_path_2)
points1 = np.array([[100, 150], [200, 250]])  # replace with proper mediapipe points
points2 = np.array([[110, 160], [210, 260]])  # random poitns

points_3d = triangulate_points(K1, R1, t1, K2, R2, t2, points1, points2)
print("Triangulated 3D points:", points_3d)


P1 = array([[ 3.80199570e+01,  1.92365636e+02, -7.08702031e+02,
         1.66744432e+06],
       [-5.54502721e+02, -1.20557929e+02, -2.45340587e+02,
         1.27083640e+06],
       [-4.31387294e-01,  7.53672921e-01, -4.95865032e-01,
         3.44408000e+03]]) 
 points1_h = array([[100, 200],
       [150, 250],
       [  1,   1]], dtype=int32)
P2 = array([[ 1.18839332e+02, -1.01970249e+02,  7.17597508e+02,
         2.47773398e+06],
       [ 4.18475729e+02, -4.50745038e+02,  4.72493837e+01,
         1.75586817e+06],
       [ 7.99632850e-01,  2.02632261e-01,  5.65267611e-01,
         5.23355000e+03]]) 
 points2_h = array([[110, 210],
       [160, 260],
       [  1,   1]], dtype=int32)
points_4d= array([[-1715583370,  1070283168],
       [-1715583370,  1070283168],
       [ 2139048840,  1072623763],
       [ 1885898724,  1069683139]], dtype=int32)
Triangulated 3D points: [[-0.90969008 -0.90969008  1.13423314]
 [ 1.00056094  1.00056094  1.00274906]]
